In [102]:
import numpy as np
from PIL import Image
import math
import matplotlib.pyplot as plt

In [103]:
path = input("Enter the Image path")
image = np.array(Image.open(path))
theta_1, theta_2 = map(lambda s: int(s), input("Enter the angles for rotation and skewness : ").split())

Enter the Image pathtest.jpg
Enter the angles for rotation and skewness : 90 45


In [104]:
def calculate_trigonometry_params(theta):
    theta = math.radians(theta)
    return math.sin(theta), math.cos(theta), math.tan(theta)

In [105]:
def skew(image, theta):
    new_height  = image.shape[0]
    tan = math.tan(math.radians(theta))
    new_width  = round(abs(image.shape[1])+abs(image.shape[0]/tan))+1
    output = np.zeros((new_height,new_width,image.shape[2]))
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):

            new_x = round(j + (i/tan))
            new_y = i

            output[new_y,new_x,:]=image[i,j,:]   
    return output


In [106]:
def rotate (image, theta):
    
    x, y, channels = image.shape[1], image.shape[0], image.shape[2]
    sin, cos, _ = calculate_trigonometry_params(theta_1)
    rotation_matrix =  np.array([[cos, -sin],
                            [sin, cos]])
    
    inverse_rotation_matrix = rotation_matrix.T
    new_height  = round(abs(y * cos) + abs(x * sin))+1
    new_width  = round(abs(x * cos) + abs(y * sin))+1
    
    output = np.zeros((new_height, new_width, channels))
    original_center_point = [round(((image.shape[1]+1)/2)-1), round(((image.shape[0]+1)/2)-1)] #x, y = width, height
    new_center_point = [round(((new_width+1)/2)-1) , round(((new_height+1)/2)-1)]
    
    
    for i in range(new_height):
        for j in range(new_width):
            
            #new grid
            y = i - new_center_point[1] +1                   
            x = j - new_center_point[0] +1

            original_x = (int(inverse_rotation_matrix[0,0] * x + inverse_rotation_matrix[0,1] * y))
            original_y = (int(inverse_rotation_matrix[1,0] * x + inverse_rotation_matrix[1,1] * y))


            new_y = original_center_point[1] + original_y
            new_x = original_center_point[0] + original_x

            if new_x < image.shape[1] and new_y < image.shape[0] and new_x>=0 and new_y>=0:
                output[i, j, :] = image[new_y, new_x, :]
    
    return output

In [107]:
def rotate_and_skew (image, theta_1, theta_2):

    output = rotate (image, theta_1)               
    output2 = skew(image, theta_2)
    output3 = skew(output, theta_2)
    
    return output, output2, output3

In [ ]:
results = rotate_and_skew(image, theta_1, theta_2)
labels = ["rotated by {}".format(theta_1), "skewed by {}".format(theta_2),
          "rotated and skewed by {} {}".format(theta_1,theta_2)]

results = list(map(lambda image:Image.fromarray((image).astype(np.uint8)), results))

for i in range(len(results)):
    results[i].save("images/"+labels[i] + ".png")
print(results)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Display the first image
axes[0,0].imshow(image)
axes[0,0].set_title('original image')

# Display the second image
axes[0,1].imshow(results[0])
axes[0,1].set_title(labels[0])

axes[1,0].imshow(results[1])
axes[1,0].set_title(labels[1])

axes[1,1].imshow(results[2])
axes[1,1].set_title(labels[2])

# Adjust the layout
plt.tight_layout()
# Show the figure
plt.show()